In [1]:

# data/yfcc100M/GT.private.2727415019.ibin
# /checkpoint/matthijs/billion-scale-ann-benchmarks/yfcc100M/GT.private.2727415019.ibin
# /checkpoint/matthijs/billion-scale-ann-benchmarks/yfcc100M/base.10M.u8bin

In [2]:
import sys
sys.path.append("/private/home/matthijs/src/big-ann-benchmarks/benchmark")

In [3]:
import dataset_io

In [11]:
import numpy as np
import faiss

In [4]:
basedir = "/checkpoint/matthijs/billion-scale-ann-benchmarks/"

gt = dataset_io.read_ibin(basedir + "yfcc100M/GT.private.2727415019.ibin")

In [6]:
xb = dataset_io.xbin_mmap(basedir + "yfcc100M/base.10M.u8bin", dtype='uint8')

In [7]:
xb.shape

(10000000, 192)

In [10]:
checksums = faiss.checksum(xb)

In [21]:
u, inv, counts = np.unique(checksums, return_inverse=True, return_counts=True)

In [22]:
counts.max()

134

In [16]:
gt_i = inv[gt]

In [49]:
# 1. if all equivalent vectors are included in the GT result list ==> no problem 
# 2. if all equivalent vectors are outside the GT result list ==> no problem either
# 3. if equivalent resutls vectors are split half-in half-out the result list ==> ambiguous

# so check cases where n_eq ids at the end of the result list that are equivalent 
# the size of the equivalence class is c >= n_eq 
# if c == n_eq ==> back to 1. 
# if n_eq < c ==> then two cases 
# * optimistic case: the user returns the selected GT vectors -- there are min(c, n_eq) of them 
# * pessimistic case: the user returns vectors that are not selected in GT -- min(n_neq, c - n_eq)

TP_count_optimistic = 0 
TP_count_pessimistic = 0 

nq = 100_000 
k = 10

for q in range(nq): 
    gt_q = gt_i[q]
    c = counts[gt_q[-1]]
    if c > 1: 
        # there is a risk!
        n_eq = (gt_q == gt_q[-1]).sum()
        assert n_eq <= c
        if n_eq < c: 
            # print(f"{q=}, {n_eq=}, {c=}")
            TP_count_optimistic += min(c, n_eq)
            TP_count_pessimistic += min(n_eq, c - n_eq)

In [50]:
(TP_count_optimistic - TP_count_pessimistic) / (nq * k)

0.000146